# Test en tiempo real

Con este notebook se pueden realizar test en tiempo real con el modelo deseado. Este notebook se ha utilizado para realziar los rest del proyecto.

El uso del notebook es sencillo:

In [ ]:
from icevision.all import *

In [2]:
from icevision.models import *

### 1. Definimos el path del modelo a cargar

In [ ]:
checkpoint_path = '/home/virtudes/SecurityApp_TFG/checkpoints/dataSetGrande.pth'

checkpoint_and_model = model_from_checkpoint(checkpoint_path)


### 2. Se cargan los datos relacionados con el modelo

In [6]:
# Just logging the info
model_type = checkpoint_and_model["model_type"]
backbone = checkpoint_and_model["backbone"]
class_map = checkpoint_and_model["class_map"]
img_size = checkpoint_and_model["img_size"]
model_type, backbone, class_map, img_size

(<module 'icevision.models.ultralytics.yolov5' from '/home/virtudes/anaconda3/envs/tfg/lib/python3.9/site-packages/icevision/models/ultralytics/yolov5/__init__.py'>,
 <ClassMap: {'background': 0, 'gun': 1}>,
 384)

In [7]:
model = checkpoint_and_model["model"]

In [8]:
device=next(model.parameters()).device
device

device(type='cuda', index=0)

In [13]:
# Transforms
img_size = checkpoint_and_model["img_size"]
valid_transforms = tfms.A.Adapter([*tfms.A.resize_and_pad(img_size), tfms.A.Normalize()])


### 3. Empezar las pruebas

Estas pruebas pueden ser realizadas con una cámara en tiempo real o con un video. Tan solo es necesario cambiar la ruta en cv2.VideoCapture('ruta'), donde ruta puede ser el path a un vídeo, el enlace a una cámara IP o un número entero (número de cámara física).

Cuando la siguiente celda se ejecuta, comienzan a contarse los fotogramas y los fotogramas con detección, además de abrirse una ventana en la que visualizar la prueba.

Para termianr la prueba, pulsar la tecla 'q'.

In [ ]:
import cv2
from PIL import Image
import numpy
capture = cv2.VideoCapture('/home/virtudes/clideo_editor_b92fb4d9587340588d179a0aa5d32a6d.mp4')
total = 0 
detected = 0
while (capture.isOpened()):
    ret, frame = capture.read()
    total+=1
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_conv = Image.fromarray(img)

    pred_dict = model_type.end2end_detect(img_conv, valid_transforms, model, class_map=class_map,
                                                      detection_threshold=0.3, return_img=False)

    if pred_dict['detection']['label_ids']:
        detected+=1
        recent_bboxes = pred_dict['detection']['bboxes']
        recent_labels = pred_dict['detection']['labels']

        for i in range(len(recent_bboxes)):
            bbox = recent_bboxes[i]
            label = recent_labels[i]
            cv2.rectangle(frame, (bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax), (255, 0, 0), 2)
            cv2.putText(frame, label, (bbox.xmin, bbox.ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    cv2.imshow('CCTV',frame)
    if (cv2.waitKey(1) == ord('q')):
        break

capture.release()
cv2.destroyAllWindows()


### Resultados

Visualizamos el número de fotogramas

In [117]:
print(detected)
print(total)

238
408


¿Cuántos fotogramas de los totales son detecciones?

In [118]:
detected/total

0.5833333333333334